# STA 220 Assignment 2

Due __Februrary 9, 2024__ by __11:59pm__. Submit your work by uploading it to Gradescope through Canvas.

Instructions:

1. Provide your solutions in new cells following each exercise description. Create as many new cells as necessary. Use code cells for your Python scripts and Markdown cells for explanatory text or answers to non-coding questions. Answer all textual questions in complete sentences.
2. The use of assistive tools is permitted, but must be indicated. You will be graded on you proficiency in coding. Produce high quality code by adhering to proper programming principles. 
3. Export the .jpynb as .pdf and submit it on Gradescope in time. To facilitate grading, indicate the area of the solution on the submission. Submissions without indication will be marked down. No late submissions accepted. 
4. If test cases are given, your solution must be in the same format. 
5. The total number of points is 10. 

__Exercise 1__

We will compute the [PageRank](https://en.wikipedia.org/wiki/PageRank) of the articles of the [Sinhala](https://en.wikipedia.org/wiki/Sinhala_language) wikipedia, which is available at [si.wikipedia.org](https://si.wikipedia.org/wiki/%E0%B6%B8%E0%B7%94%E0%B6%BD%E0%B7%8A_%E0%B6%B4%E0%B7%92%E0%B6%A7%E0%B7%94%E0%B7%80). Additional information of the Sinhala wiki can be found [here](https://meta.wikimedia.org/wiki/List_of_Wikipedias). 

_Hints: If you don't speak Sinhalese, you might want to learn the wiki logic from the english wikipedia, and translate your findings. Also, caching is highly recommended._

__(a)__ Use the special [AllPages](https://si.wikipedia.org/wiki/%E0%B7%80%E0%B7%92%E0%B7%81%E0%B7%9A%E0%B7%82:%E0%B7%83%E0%B7%92%E0%B6%BA%E0%B7%85%E0%B7%94_%E0%B6%B4%E0%B7%92%E0%B6%A7%E0%B7%94) page and understand its logic to retrieve the url of all articles in the sinhalese wikipedia. Make sure to skip redirections.

_How many articles are there?_

In [3]:
#(a)
import requests
import lxml.html as lx
import requests_cache
requests_cache.install_cache('wiki_cache')

In [4]:
article_urls= []
base_url = "https://si.wikipedia.org"

def get_all_article_urls(base_url):
    
    # first page
    current_url='https://si.wikipedia.org/w/index.php?title=විශේෂ:සියළු_පිටු&from=%22අන්තර්+ජාලය+තුල%2C+ඔබ+බල්ලෙකු+බව+කිසිවෙක්+නොදනි'
    response = requests.get(current_url)
    tree = lx.fromstring(response.content)
    articles = tree.xpath('//*[@id="mw-content-text"]/div[3]//a[not(contains(@class, "mw-redirect"))]/@href') #skip redirect
    for article in articles:
        urls = [base_url + article]
        article_urls.append(urls)
    

    # second to last 
    current_url = base_url + "/w/index.php?title=විශේෂ:සියළු_පිටු&from=1880+ගණන්"
  
    while current_url:
        response = requests.get(current_url)
        tree = lx.fromstring(response.content)
        
        # Extract article URLs from the current page
        articles = tree.xpath('//*[@id="mw-content-text"]/div[3]//a[not(contains(@class, "mw-redirect"))]/@href') #skip redirect
        for article in articles:
            urls = [base_url + article]
            article_urls.append(urls)
        
        # Check for the next page link
        next_page=tree.xpath('//*[@id="mw-content-text"]/div[2]/a/@href')
        try:
            if next_page:
                current_url = base_url + next_page[1]
            else:
                current_url = None
        except IndexError:
            print("Reached last page")
            current_url = None

    return article_urls


In [5]:
# a 
get_all_article_urls(base_url)
flat_total_article= [item for sublist in article_urls for item in sublist]

Reached last page


In [84]:
#a
print(f'There are total {len(flat_total_article)} pages without redirections')

There are total 24233 pages without redirections


__(b, i)__ Scan all articles in the sinhalese wikipedia and retrieve all links to other articles. Avoid links to special pages, images or the ones that point to another website. Only count the proper article for links that point to a specific section. Use regular expressions to manage these cases. 
__(ii)__ Make sure to match redirections to their correct destiation article. To this end, find how wikipedia treats redirections and retrieve the true article. _(Help: Try searching for 'uc davis' on en.wikipedia.org')_
__(iii)__ Use threading to request all articles and obtain all links to other articles. _(Attention: This takes about thirty minutes!)_


_How many links to other articles are there?_

In [81]:
#(b,i) solution
import requests
from bs4 import BeautifulSoup
import re

def get_all_article_links(urls):
    article_links = {}
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        content_div = soup.find(id='mw-content-text')
        if content_div:
            links = content_div.find_all('a', href=True)
            link_list = []
            for link in links:
                href = link['href']
                # Use regular expressions to filter out unwanted links
                if not re.match(r'^/wiki/(?!Special:|%[A-F0-9]{2}).*$', href) \
                        and not re.match(r'^https?://.+\.wikipedia\.org/wiki/.+$', href) \
                        and not re.search(r'\.(jpg|jpeg|png|gif|svg)$', href) \
                        and not re.search(r':Cite', href) \
                        and not re.match(r'^/wiki/File:.*$', href) \
                        and not re.search(r'action=edit|redlink=1', href) \
                        and re.match(r'^/wiki/.+', href) \
                        and not '#' in href:
                    link_list.append(base_url+href)
            article_links[url] = link_list
    return article_links


In [97]:
#(b,ii) solution

def filter_existing_links(existing_links, article_links):
    updated_links = {}

    for url, links in existing_links.items():
        # Filter links that are present in article_links
        filtered_links = [link for link in links if link in article_links]
        # Check if any filtered links are present
        if filtered_links:
            updated_links[url] = filtered_links
    return updated_links


In [16]:
#(b,iii)

article_links=get_all_article_links(flat_total_article)

In [114]:
#(b,iii) 
updated_links=filter_existing_links(article_links,flat_total_article)

In [131]:
#(b,iii) solution ,Kernel dies when using threading 
total_values = sum(len(value) for value in updated_links.values())
print("Total number of links to other articles:", total_values)

Total number of links to other articles: 303237


In [ ]:
#(b,iii) code with threading 
from concurrent.futures import ThreadPoolExecutor
def process_links(urls):
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = executor.map(get_all_article_links, urls)

article_links_dict = process_links()



__(c)__ Compute the transition matrix (see [here](https://en.wikipedia.org/wiki/Google_matrix) and [here](https://www.amsi.org.au/teacher_modules/pdfs/Maths_delivers/Pagerank5.pdf) for step-by-step instructions). Make sure to tread dangling nodes. You may want to use: 
```
from scipy.sparse import csr_matrix
```

In [129]:
#(c) solution
import numpy as np
from scipy.sparse import csr_matrix

def compute_transition_matrix(links):
    all_pages = set(links.keys())
    linked_pages = set(page for linked_pages in links.values() for page in linked_pages)
    dangling_nodes = all_pages - linked_pages
    num_pages = len(all_pages)
    transition_matrix = np.zeros((num_pages, num_pages))
       
    for i, page in enumerate(all_pages):
        num_outlinks = len(links[page])
        if num_outlinks == 0:  
            transition_matrix[i] = 1 / num_pages
        else:
            for linked_page in links[page]:
                if linked_page in all_pages:  
                    j = list(all_pages).index(linked_page)
                    transition_matrix[i][j] = 1 / num_outlinks
    

    return csr_matrix(transition_matrix)


transition_matrix = compute_transition_matrix(updated_links)
print(transition_matrix.toarray())


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


__(d, i)__ Set the damping factor to `0.85` and compute the PageRank for each article, using fourty iterations and starting with a vector with equal entries. __(ii)__ Obtain the top ten articles in terms of PageRank, and, retrieving the articles again, find the correponding English article, if available. 

_Return the corresponding English article titles of the top ten articles from the Sinhalese wikipedia._

In [141]:
# (d,i) solution

def compute_pagerank(transition_matrix, damping_factor=0.85, num_iterations=40):
    num_pages = transition_matrix.shape[0]
    pagerank = np.ones(num_pages) / num_pages

    for _ in range(num_iterations):
        pagerank = (1 - damping_factor) / num_pages + damping_factor * transition_matrix.dot(pagerank)

    return pagerank

pagerank = compute_pagerank(transition_matrix, damping_factor=0.85, num_iterations=40)
print(pagerank)

[2.73616882e-05 3.67762696e-05 2.55561583e-05 ... 8.75401226e-06
 3.53263591e-05 2.47862561e-05]


In [152]:
# (d,ii) solution

def top_10_page_urls(pagerank_scores, urls):
    pagerank_urls = zip(pagerank_scores, urls)
    sorted_pagerank_urls = sorted(pagerank_urls, reverse=True)
    top_10_urls = [url for _, url in sorted_pagerank_urls[:10]]
    return top_10_urls

pagerank_scores = compute_pagerank(transition_matrix)
top_10_urls = top_10_page_urls(pagerank_scores, list(updated_links.keys()))

for url in top_10_urls:
    print(url)


https://si.wikipedia.org/wiki/Sum
https://si.wikipedia.org/wiki/Scutigera_coleoptrata
https://si.wikipedia.org/wiki/Pseudophilautus_asankai
https://si.wikipedia.org/wiki/992
https://si.wikipedia.org/wiki/5_%E0%B7%80%E0%B6%B1_%E0%B6%86%E0%B7%80%E0%B6%BB%E0%B7%8A%E0%B6%AD%E0%B6%BA%E0%B7%9A_%E0%B6%B8%E0%B7%96%E0%B6%BD%E0%B6%AF%E0%B7%8A%E2%80%8D%E0%B6%BB%E0%B7%80%E0%B7%8A%E2%80%8D%E0%B6%BA%E0%B6%BA
https://si.wikipedia.org/wiki/2021_%E0%B6%BA%E0%B7%94%E0%B6%BB%E0%B7%9D%E0%B6%B4%E0%B7%92%E0%B6%BA%E0%B7%8F%E0%B6%B1%E0%B7%94_%E0%B6%A2%E0%B6%BD%E0%B6%9C%E0%B7%90%E0%B6%BD%E0%B7%94%E0%B6%B8%E0%B7%8A
https://si.wikipedia.org/wiki/1812_%E0%B6%BA%E0%B7%94%E0%B6%AF%E0%B7%8A%E0%B6%B0%E0%B6%BA
https://si.wikipedia.org/wiki/150_%E0%B6%B8%E0%B7%94%E0%B6%BD%E0%B7%8A%E0%B6%BD%E0%B7%9A%E0%B6%BB%E0%B7%92%E0%B6%BA%E0%B7%8F%E0%B7%80
https://si.wikipedia.org/wiki/13
https://si.wikipedia.org/wiki/%E0%B7%86%E0%B7%92%E0%B6%B6%E0%B7%9C%E0%B6%B1%E0%B7%8F%E0%B6%A0%E0%B7%8A%E0%B6%A0%E0%B7%92_%E0%B7%83%E0%B6%82%E0%B6%